In [1]:
#Criado por: Vinícius de Almeida Nery Ferreira (ECO - UnB)
#Github: https://github.com/vnery5/Econometria

#######################################################################################################################
###COMO USAR AS FUNÇÕES EM UM NOTEBOOK
##Antes, copie e cole todos os imports e definições daqui na primeira célula do notebook e pressione Shift + Enter
##Para coletar os dados do arquivo "carros.dta" (só funciona com arquivos .dta):
#coletar_dados("carros")

## Exportar resultados como imagem ou texto: https://stackoverflow.com/questions/46664082/python-how-to-save-statsmodels-results-as-image-file
#######################################################################################################################

##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats

#Para Regressão Linear Múltipla (OLS, GLS e WLS) e Testes Estatísticos
import statsmodels.api as sm
import econtools
import econtools.metrics as mt

#Para Regressão em Painel
from linearmodels import PanelOLS, FirstDifferenceOLS, PooledOLS, RandomEffects
from linearmodels.panel import compare

#Pacotes para gráficos (caso precise)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output
import gc
import subprocess #permite a cópia para o clipboard das equações gerados com as funções equation()

####################################### Criando as Funções ###############################################################

def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata (.dta) - NÃO COLOQUE A EXTENSÃO NA HORA DE NOMEAR O "NOME"!
    O arquivo deve estar na mesma pasta do arquivo de Python ou do notebook do jupyter.
    Deixe em branco para ler o arquivo mais recentemente adicionado à pasta.
    '''

    global df

    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #No meu caso específico:
    caminho_vinicius = f"{caminho}/datasets"

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        try:
            arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
        except:
            arquivo = max(glob.glob(f"{str(caminho_vinicius)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except:
            try:
                arquivo = f"{str(caminho_vinicius)}/{str(nome)}.dta"
                df = pd.read_stata(arquivo)
                print(f"{nome}.dta foi lido com sucesso!")
                return df
            except: #caso não tenha sido encontrado o arquivo com o nome inserido
                print('''
                Não foi possível achar o arquivo :(\n
                Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
                ''')

def Regressao_Multipla(x, y, constante = "S", cov = "normal"):
    '''
    Função que calcula uma regressão múltipla, sendo, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    cov: "normal" para regressão com erros-padrão tradicionais (caso padrão);
        "robust" para erros-padrões robustos.
        "cluster" ou "clustered" para erros-padrões clusterizados
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares (OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x

    #Criando o Modelo levando em conta a opção de erros padrão
    Modelo = sm.OLS(y,X)

    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    elif cov == "cluster" or cov == "clustered":
        group = str(input("Qual o rótulo da coluna de grupo?"))
        try:
            Resultado = Modelo.fit(cov_type = 'cluster',cov_kwds  ={'groups':df[group]}, use_t = True)
        except:
            erro = "Não foi possível encontrar o grupo selecionado. Tente novamente!"
            return erro
    else:
        Resultado = Modelo.fit()
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(Resultado.summary())

    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}")
    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade.
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )

def Regressao_MQP(x, y, pesos, constante = "S", cov = "normal"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados ponderados, ou seja,
    recomendada quando o erro é heteroscedástico E se sabe a função da constante. Ela é, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    pesos: 1/h, sendo h a constante multiplicativa da variância do erro (ou seja, sem a raiz);
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    cov: "normal" para regressão com erros-padrão tradicionais (caso padrão);
        "robust" para erros-padrões robustos.
        "cluster" para erros-padrões clusterizados
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x

    #Criando o Modelo levando em conta a opção de erros padrão
    Modelo = sm.WLS(y,X, weights = pesos)

    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    elif cov == "cluster" or cov == "clustered":
        group = str(input("Qual o rótulo da coluna de grupo?"))
        try:
            Resultado = Modelo.fit(cov_type = 'cluster',cov_kwds  ={'groups':df[group]}, use_t = True)
        except:
            erro = "Não foi possível encontrar o grupo selecionado. Tente novamente!"
            return erro
    else:
        Resultado = Modelo.fit()

    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )
    
def Regressao_MQGF(x, y, constante = "S", cov = "normal"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados generalizados factíveis, ou seja,
    recomendada quando o erro é heteroscedástico E NÃO se sabe a função da constante multiplicativa da variância do erro, sendo os pesos estimados
    regridindo o log dos quadrados dos resíduos sobre as variáveis explicativas. Os estimadores MQP são gerados com o peso estimado.
    Ela é, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    cov: "normal" para regressão com erros-padrão tradicionais (caso padrão);
        "robust" para erros-padrões robustos.
        "cluster" ou "clustered" para erros-padrões clusterizados
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #Regredindo os valores normalmente a fim de pegar os resíduos
    Regressao_Multipla(x,y, constante, cov)
    clear_output()

    #Coletando o log dos quadrados dos resíduos
    Log_Res_Quad = np.log(Resíduos**2)

    #Regredindo Log_Res_Quad sobre as variáveis explicativas
    Regressao_Multipla(x,Log_Res_Quad, constante, cov)
    clear_output()

    #Estimando os pesos
    Pesos = np.exp(Lista_ychapeu)

    #Fazendo uma Regressão MQP
    Regressao_MQP(x,y, 1/Pesos, constante, cov)

def Teste_LM(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste LM e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)
    
    #Fazendo a regressão do modelo restrito e armazenando os resíduos
    Regressao_Multipla(df[ModeloRestrito], y)
    Resíduos_r = Resíduos

    #Fazendo a regressão dos resíduos sobre as variáveis independentes e armazenando o R2
    Regressao_Multipla(x, Resíduos_r)
    Ru = Resultado.rsquared

    #Calculando a estatística LM
    LM = Numero_de_Observações*Ru

    #Calculando o p-valor
    ##Calculando o P-valor de F
    P_valor = stats.chi2.sf(LM,GL_r)

    #Limpando a tela
    clear_output()

    #Printando o resultado
    if Nivel_de_Significância > P_valor:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")


def Teste_F(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    #para testar igualdade dos coeficientes, F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressao_Multipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressao_Multipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho à significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho à significância de {Nivel_de_Significância*100}%.")

def Teste_F_Rapido_Robusto(H0, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F de forma mais rápida com base nas restrições de H0, podendo ser robusto se o Resultado for fruto de uma regressão robusta.
    H0 deve estar na forma B1 = B2 =...= Valor que deseja ser testado (0 na maioria das vezes)
    '''
    global Resultado
    ## A função utiliza o método wald_test dos resultados das regressões
    # Para modelos de painel - cujo método usa a estatística LM -, devemos especificar o parâmetro 'formula', o que não ocorre com cortes transversais
    try:
        teste = 'LM'
        est = Resultado.wald_test(formula=H0).stat
        p = Resultado.wald_test(formula=H0).pval
    except:
        teste = 'F'
        est = float(str(Resultado.wald_test(H0))[19:29])
        p = float(str(Resultado.wald_test(H0))[36:47])

    if Nivel_de_Significância > p:
        print(f"O valor de {teste} é {round(est,6)} e seu p-valor é {round(p,7)}.\nPortanto, rejeita-se Ho à significância de {Nivel_de_Significância*100}%, ou seja, as variáveis são conjuntamente significantes.")
    else:
        print(f"O valor de {teste} é {round(est,6)} e seu p-valor é {round(p,7)}.\nPortanto, NÃO se rejeita Ho à significância de {Nivel_de_Significância*100}%, ou seja, as variáveis NÃO são conjuntamente significantes.")

def Teste_t_Dois_Coeficientes_Iguais(x, y, Coeficientes_Testados_para_serem_iguais, Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste t para verificar se dois coeficientes são iguais.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Coeficientes_Testados_para_serem_iguais: array com os valores dos coeficientes que querem ser testados;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    ##Fazendo a regressão do modelo irrestrito
    Regressao_Multipla(x, y)
    clear_output()

    #Fazendo o objeto de lista que será usado no teste
    Teste =[0]
    Num_de_Variaveis = 1

    for i in list(x):
        if i not in list(Coeficientes_Testados_para_serem_iguais):
            Teste.append(0)
        elif (Num_de_Variaveis % 2 == 0):
            Teste.append(-1)
        else:
            Teste.append(1)
            Num_de_Variaveis += 1

    Teste_t = Resultado.t_test(Teste)
    print(f"A estatística do teste é {Teste_t.tvalue}, o que resulta em um p-valor bilateral de {Teste_t.pvalue} e em um p-valor unilateral de {Teste_t.pvalue/2}.")

def Teste_Heteroscedasticidade_BP(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de Breusch-Pagan para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística = LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressao_Multipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resíduos**2

    #Realizando o teste F ou LM de Res_Quad sobre as variaveis dependentes para ver se há correlação
    if Estatística == "LM":
        Teste_LM(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def Teste_Heteroscedasticidade_White(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de White (modificado por Wooldridge) para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística: LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressao_Multipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resultado.resid**2

    #Calculando o quadrado dos valores previstos
    Previstos = Lista_ychapeu
    Previstos2 = Previstos**2

    #Criando um dataframe pra armazenar esses valores
    dfy_y2 = pd.DataFrame({"y":Previstos,"y2":Previstos2})
    y_y2 = dfy_y2[['y','y2']]

    #Realizando o teste F ou LM de Res_Quad sobre y e y^2
    if Estatística == "LM":
        Teste_LM(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def RESET(x, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste RESET para verificar a adequação das formas funcionais.
    Ho: o modelo está bem especificado.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    #Fazendo uma regressão múltipla e limpando a tela
    Regressao_Multipla(x, y, constante)
    clear_output()

    #Verificando o tipo da covariância selecionada
    if robusta == "N":
        tipo = 'nonrobust'
    else:
        tipo = 'HC1'

    Teste = sm.stats.diagnostic.linear_reset(Resultado, power = 2, use_f = False, cov_type = tipo)
    
    if Teste.pvalue < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (o modelo está MAL especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, maior que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (o modelo NÃO está MAL especificado)"""
        )

def Teste_J_Davidson_MacKinnon(x1,x2, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste J para verificar qual o modelo mais adequado (dentre os dois colocados).
    Ho: o modelo 1 é preferível (ver o p-valor do último coeficiente).

    x1: lista ou array com os valores das variáveis independentes do primeiro modelo;
    x2: lista ou array com os valores das variáveis independentes do segundo modelo;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    #Fazendo a regressão do segundo modelo
    Regressao_Multipla(x2, y, constante, robusta)
    clear_output()

    #Criando um novo dataframe e adicionando os valores previstos do modelo 2 à x
    Valores_Previstos_2 = pd.DataFrame({'Previsão M1':Lista_ychapeu})
    x = pd.concat([x1, Valores_Previstos_2], axis=1, sort=False)

    #Fazendo a regressão do primeiro modelo sobre x
    Regressao_Multipla(x, y, constante, robusta)
    clear_output()

    #Pegando o p-valor do teste
    P_valor = Resultado.pvalues[-1]

    if P_valor < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (ou seja, o modelo 2 ({list(x2)}) é mais bem especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (ou seja, o modelo 1 ({list(x1)}) é mais bem especificado)."""
        )

######### Funções de Dados em Painel #########
def Arrumar_Painel():
    '''
    Função que transforma o painel num formato que o PanelOLS consegue ler (index multinível e coluna do tipo categoria para os anos)
    '''
    global df

    # pedir a coluna com os indivíduos; se o nome for inválido, sair da função.
    coluna_individuos = str(input('Qual o rótulo da coluna de indivíduos/clusters?\n'))
    if coluna_individuos not in df.columns:
        print("Coluna de indivíduos/clusters não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None
    
    # pedir a coluna com os períodos de tempo; se o valor for inválido, sair da função.
    coluna_tempo = str(input('Qual o rótulo da coluna de tempo/observações dos clusters?\n'))
    if coluna_tempo not in df.columns:
        print("Coluna de tempo/observações não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None

    ## arrumando o painel
    periodos = pd.Categorical(df[coluna_tempo])
    df = df.set_index([coluna_individuos,coluna_tempo])
    df[coluna_tempo] = periodos
    return df

def Reg_Painel_Primeiras_Diferenças (x,y, cov = "normal"):
    '''
    Função que calcula uma regressão de primeiras diferenças SEM um intercepto, sendo, por default, computada com erros padrões não robustos.
    Para calcular a regressão com um intercepto, ver o notebook "Cap 13 e 14".
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    cov: "normal" para regressão com erros-padrão tradicionais (caso padrão);
        "robust" para erros-padrões robustos.
        "cluster" para erros-padrões clusterizados
    '''
    global df, Resultado

    Modelo = FirstDifferenceOLS(y, x)

    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'robust')
    elif cov == 'kernel': ## correlação robusta à heteroscedasticidade e autocorrelação serial
        Resultado = Modelo.fit(cov_type = 'kernel')
    elif cov == 'clustered' or cov == 'cluster':
        Resultado = Modelo.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        Resultado = Modelo.fit()

    print(Resultado)

def Reg_Painel_Efeitos_Fixos(x, y, constante = "S", cov='normal'):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões não robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    cov: "normal" para regressão com erros-padrão tradicionais (caso padrão);
        "robust" para erros-padrões robustos.
        "cluster" ou "clustered" para erros-padrões clusterizados
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção dos erros padrão
    Modelo = PanelOLS(y,X, entity_effects=True, drop_absorbed=True)

    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'robust')
    elif cov == 'kernel': ## correlação robusta à heteroscedasticidade e autocorrelação serial
        Resultado = Modelo.fit(cov_type = 'kernel')
    elif cov == 'clustered' or cov == 'cluster':
        Resultado = Modelo.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        Resultado = Modelo.fit()

    print(Resultado)

def Reg_Painel_MQO_Agrupado(x, y, constante = "S", cov = "normal"):
    '''
    Função que calcula uma regressão por MQO agrupado, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    cov: "normal" para regressão com erros-padrão tradicionais (caso padrão);
        "robust" para erros-padrões robustos.
        "cluster" ou "clustered" para erros-padrões clusterizados
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção do erro padrão
    Modelo = PooledOLS(y,X)

    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'robust')
    elif cov == 'kernel': ## correlação robusta à heteroscedasticidade e autocorrelação serial
        Resultado = Modelo.fit(cov_type = 'kernel')
    elif cov == 'clustered' or cov == 'cluster':
        Resultado = Modelo.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        Resultado = Modelo.fit()
    print(Resultado)

def Reg_Painel_Efeitos_Aleatórios(x, y, constante = "S", cov = "normal"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo
    Modelo = RandomEffects(y,X)
    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'robust')
    elif cov == 'kernel': ## correlação robusta à heteroscedasticidade e autocorrelação serial
        Resultado = Modelo.fit(cov_type = 'kernel')
    elif cov == 'clustered' or cov == 'cluster':
        Resultado = Modelo.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        Resultado = Modelo.fit()
    print(Resultado)

def hausman_EF_EA(x_inef, y, Nivel_de_Significância = 0.05):
    '''
    Função que faz um teste de Hausman, em que H0: Não há correlação entre os efeitos não-observados e as variáveis explicativas
    x_inef: variáveis explicativas do modelo ineficiente sob H0 (EF);
    y: variável explicativa
    '''
    ## Fazendo a regressão de efeitos fixos e guardando o resultado
    Reg_Painel_Efeitos_Fixos(x,y)
    clear_output()
    fixed = Resultado

    ## Fazendo a regressão de efeitos aleatórios e guardando o resultado
    Reg_Painel_Efeitos_Aleatórios(x,y)
    clear_output()
    random = Resultado

    ## Calculando a estatística de Hausman
    # calculando a diferença entre os parametros e a variância assíntótica da diferença entre os parametros
    var_assin = fixed.cov - random.cov
    d = fixed.params - random.params
    
    # calculando H
    H = d.dot(np.linalg.inv(var_assin)).dot(d)
    # calculando os graus de liberdade
    gl = random.params.size -1
    # Calculando o P-valor do teste
    p = stats.chi2(gl).sf(H)

    if p < Nivel_de_Significância:
        print(f"O valor de H é {round(H,6)} com {gl} graus de liberdade na distribuição chi2. O p-valor do teste é {round(p,6)} e, portanto, se rejeita H0 e prefere-se o modelo de efeitos fixos.")
    else:
        print(f"O valor de H é {round(H,6)} com {gl} graus de liberdade na distribuição chi2. O p-valor do teste é {round(p,6)} e, portanto, não se rejeita H0 e prefere-se o modelo de efeitos aleatórios.")

def equation(sep_erros= "("):
    '''
    Função que gera uma equação formatada do word
    '''
    
    ## capturando os parametros e os erros
    params = dict(np.around(Resultado.params,3))
    ## linearmodels usa .std_erros para capturar os erros padrão, sm usa .bse
    try:
        std_errors = dict(np.around(Resultado.std_errors,5))
    except:
        std_errors = dict(np.around(Resultado.bse,5))

    ## fazendo o loop para pegar os coeficientes*nome das variáveis e os seus erros-padrão entre parênteses
    parametros = ""
    erros = ""
    for i in params.keys():
        # levando em conta a chave escolhida pelo usuário
        if sep_erros == "(":
            erros += f" & ({std_errors[i]})"
        else:
            erros += f" & [{std_errors[i]}]"

        if i != 'const':
            if params[i] > 0:
                parametros += f" & + {params[i]}{i}"
            else:
                parametros += f" & - {-params[i]}{i}"
        else:
            parametros += f"{params[i]}"
    
    ## Fazendo a str que irá pro word (em forma de matriz)
    inicio = "\matrix{"
    fim = "}"

    # linearmodels usa model.dependente; sm usa model.endog_names
    try:
        y = Resultado.model.dependent.dataframe.columns[0]
    except:
        y = Resultado.model.endog_names
    word = f"{inicio}{y} & = & {parametros} \\\ & {erros}{fim}"

    ## Adicionando o numero de obs e os r2
    # linearmodels usa .entity_info['total'] no numero de observações, sm usa nobs
    try:
        word += f"\nn = {int(dict(Resultado.entity_info)['total'])}; R^2 = {np.around(Resultado.rsquared,3)}"
    except:
         word += f"\nn = {int(Resultado.nobs)}; R^2 = {np.around(Resultado.rsquared,3)}; R\\bar^2 = {np.around(Resultado.rsquared_adj,3)}"

    ## substituindo os . por ,
    word = word.replace(".",",")
    
    ## copiando para o clipboard e printando o sucesso
    subprocess.run("pbcopy", universal_newlines=True, input=word)
    print("O código da equação foi copiado para o clipboard!")

## Exemplos Cap. 15

In [6]:
from linearmodels.iv import IV2SLS
coletar_dados('MROZ')

MROZ.dta foi lido com sucesso!


,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,0,0,0,2,40,13,NaN,0.00,3020,43,...,28200.0,0.6215,10,10,9.5,1,5,28.200001,NaN,25
749,0,0,2,3,31,12,NaN,0.00,2056,33,...,10000.0,0.7715,12,12,7.5,0,14,10.000000,NaN,196
750,0,0,0,0,43,12,NaN,0.00,2383,43,...,9952.0,0.7515,10,3,7.5,0,4,9.952000,NaN,16
751,0,0,0,0,60,12,NaN,0.00,1705,55,...,24984.0,0.6215,12,12,14.0,1,15,24.983999,NaN,225


In [8]:
## limpando os nans
df.dropna(inplace=True, subset=['lwage','educ'], axis = 0)
df

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,1,680,0,5,36,10,2.3118,0.00,3430,43,...,19772.0,0.7215,7,7,7.5,0,2,18.199976,0.838026,4
424,1,2450,0,1,40,12,5.3061,6.50,2008,40,...,35641.0,0.6215,7,7,5.0,1,21,22.641056,1.668857,441
425,1,2144,0,2,43,13,5.8675,0.00,2140,43,...,34220.0,0.5815,7,7,7.5,1,22,21.640079,1.769429,484
426,1,1760,0,1,33,12,3.4091,3.21,3380,34,...,30000.0,0.5815,12,16,11.0,1,14,23.999985,1.226448,196


In [9]:
## Fazendo a estimação por MQO
y = df['lwage']
x = df[['educ']]
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     56.93
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           2.76e-13
Time:                        09:28:40   Log-Likelihood:                -441.26
No. Observations:                 428   AIC:                             886.5
Df Residuals:                     426   BIC:                             894.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1852      0.185     -1.000      0.3

In [11]:
## obtendo a eq reduzida com educp como instrumento
x = df[['fatheduc']]
y = df['educ']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     88.84
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           2.76e-19
Time:                        09:29:55   Log-Likelihood:                -920.02
No. Observations:                 428   AIC:                             1844.
Df Residuals:                     426   BIC:                             1852.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.2371      0.276     37.099      0.0

In [12]:
## Regredindo lwage sobre educ_chapeu
x = Lista_ychapeu
y = df['lwage']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.586
Date:                Thu, 25 Mar 2021   Prob (F-statistic):              0.109
Time:                        09:30:32   Log-Likelihood:                -466.81
No. Observations:                 428   AIC:                             937.6
Df Residuals:                     426   BIC:                             945.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4411      0.467      0.944      0.3

In [25]:
## Fazendo uma regressão por MQ2E
exog = df[['exper','expersq']]
exog = sm.add_constant(exog)
y = df['lwage']
endog = df[['educ']]
instrumentos = df[['motheduc','fatheduc']]

modelo = IV2SLS(y,exog,endog,instrumentos)
resultado = modelo.fit()
print(resultado)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    18.611
Date:                Thu, Mar 25 2021   P-value (F-stat)                0.0003
Time:                        09:46:17   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0481     0.4278     0.1124     0.9105     -0.7903      0.8865
exper          0.0442     0.0155     2.8546     0.00

In [26]:
## vendo as estatísticas da equação reduzida/1º estágio
# note que IV2SLS pode ser igual à MQO sendo endog e instrumentos igual a None
print(resultado.first_stage)

    First Stage Estimation Results    
                                  educ
--------------------------------------
R-squared                       0.2115
Partial R-squared               0.2076
Shea's R-squared                0.2076
Partial F-statistic             100.22
P-value (Partial F-stat)        0.0000
Partial F-stat Distn           chi2(2)
========================== ===========
const                           9.1026
                              (21.588)
exper                           0.0452
                              (1.0854)
expersq                        -0.0010
                             (-0.7671)
motheduc                        0.1576
                              (4.4718)
fatheduc                        0.1895
                              (5.8771)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [27]:
## Testando para a exogeneidade da variável instrumentos
resultado.wooldridge_regression
# o p-valor do teste robusto é 0,1081, de modo que não rejeitamos a hipotese de exogeneidade das variáveis instrumentadas

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 2.5818
P-value: 0.1081
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fb959363750

In [28]:
## testando para restrições sobreidentificadores (se os intrumentos são realmente exógenos)
# como temos dois instrumentos para educ, podemos realizar o teste
resultado.wooldridge_overid
# o p-valor é >0,5; não se rejeita h0 (os intrumentos são realmente exógenos)

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 0.4435
P-value: 0.5055
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fb95c6c7a50

In [2]:
def Regressao_IV_MQ2E(exog, endog, instrumentos, y, constante="S",cov='normal'):
    global df, Resultado
    ## formando o vetor de variáveis exógenas
    if constante == "S":
        try:
            exog = sm.add_constant(exog)
        except Exception: ## se não houver exógenas no modelo
            exog = np.resize([1],endog.shape[0])
            exog = pd.DataFrame({'const':exog})
    else:
        exog = exog
    
    ## criando o modelo levando em conta a opção de covariância
    Modelo = IV2SLS(y,exog,endog,instrumentos)
    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'robust')
    elif cov == 'kernel': ## correlação robusta à heteroscedasticidade e autocorrelação serial
        Resultado = Modelo.fit(cov_type = 'kernel')
    elif cov == 'clustered' or cov == 'cluster':
        Resultado = Modelo.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        Resultado = Modelo.fit(cov_type='unadjusted')
    
    print(Resultado)
    print("\nPara ver os resultados do 1º estágio/equação reduzida (e ver se os instrumentos são relevantes, chame 'Resultado.first_stage'.")
    print("\nPara testar a exogeneidade da variável instrumentada, chame 'Resultado.wooldridge_regression' ou 'Resultado.wooldridge_score' ou 'Resultado.wu_hausman([variaveis])'.")
    print("\nPara testar a exogeneidade dos instrumentos (quando eles forem mais numerosos que as variáveis endógenas), chame 'Resultado.wooldridge_overid', onde Ho: todos os instrumentos são exógenos.")

In [3]:
def equation(sep_erros= "("):
    '''
    Função que gera uma equação formatada do word
    '''
    
    ## capturando os parametros, p-valores e os erros
    params = dict(np.around(Resultado.params,3))
    p_values = dict(np.around(Resultado.pvalues,4))

    ## linearmodels usa .std_erros para capturar os erros padrão, sm usa .bse
    try:
        std_errors = dict(np.around(Resultado.std_errors,5))
    except Exception:
        std_errors = dict(np.around(Resultado.bse,5))

    ## fazendo o loop para pegar os coeficientes*nome das variáveis 
    # e os seus erros-padrão e p-valores entre parênteses
    parametros = ""
    erros = ""
    p = ""
    for i in params.keys():
        # levando em conta a chave escolhida pelo usuário
        if sep_erros == "(":
            erros += f" & ({std_errors[i]})"
            p += f" & ({p_values[i]})"
        else:
            erros += f" & [{std_errors[i]}]"
            p += f" & [{p_values[i]}]"

        if i != 'const':
            if params[i] > 0:
                parametros += f" & + {params[i]}{i}"
            else:
                parametros += f" & - {-params[i]}{i}"
        else:
            parametros += f"{params[i]}"
    
    ## Fazendo a str que irá pro word (em forma de matriz)
    inicio = "\matrix{"
    fim = "}"

    ## pegando o nome da variável dependente
    # linearmodels usa model.dependent.dataframe.columns[0] para modelos de painel 
    # e model.dependent.cols para modelos de IV;
    # sm usa model.endog_names
    try:
        try: # modelos de painel
            y = Resultado.model.dependent.dataframe.columns[0]
        except Exception: # modelos de IV 2SLS
            y = Resultado.model.dependent.pandas.columns[0]
    except Exception: # modelos comuns
        y = Resultado.model.endog_names

    word = f"{inicio}{y} & = & {parametros} \\\ erros\_padrão & = {erros} \\\ p\_valores & = {p}{fim}"

    ## Adicionando o numero de obs e os r2 (quando rsquared_adj estiver disponível)
    try:
        word += f"\nn = {int(Resultado.nobs)}; R^2 = {np.around(Resultado.rsquared,4)}; \\bar{{R^2}} = {np.around(Resultado.rsquared_adj,4)}"
    except Exception:
        word += f"\nn = {int(Resultado.nobs)}; R^2 = {np.around(Resultado.rsquared,4)}"

    ## substituindo os . por ,
    word = word.replace(".",",")
    
    ## copiando para o clipboard e printando o sucesso
    subprocess.run("pbcopy", universal_newlines=True, input=word)
    print("O código da equação foi copiado para o clipboard!")

In [46]:
## Fazendo uma regressão por MQ2E
exog = df[['exper','expersq']]
exog = sm.add_constant(exog)
y = df['lwage']
endog = df[['educ']]
instrumentos = df[['motheduc','fatheduc']]

Regressao_IV_MQ2E(exog,endog,instrumentos,y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    24.653
Date:                Thu, Mar 25 2021   P-value (F-stat)                0.0000
Time:                        10:16:03   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0481     0.3985     0.1207     0.9039     -0.7329      0.8291
exper          0.0442     0.0134     3.3038     0.00

In [47]:
## Podemos, por último, combinar PD e IV ('PANEL IV'): https://bashtage.github.io/linearmodels/iv/examples/basic-examples.html

## Exercício C1

In [167]:
coletar_dados('WAGE2')

WAGE2.dta foi lido com sucesso!


,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,520,40,79,28,16,6,1,30,1,1,1,0,0,1.0,11.0,NaN,6.253829
931,1202,40,102,32,13,10,3,31,1,0,1,1,7,7.0,8.0,6.0,7.091742
932,538,45,77,22,12,12,10,28,1,1,1,0,9,NaN,7.0,NaN,6.287858
933,873,44,109,25,12,12,12,28,1,0,1,0,1,1.0,NaN,11.0,6.771935


In [168]:
## o modelo do exemplo 15.2 é
x = df[['educ']]
y = df['lwage']

Regressao_IV_MQ2E(exog= x, endog= None, instrumentos= None, y=y)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.0974
Estimator:                        OLS   Adj. R-squared:                 0.0964
No. Observations:                 935   F-statistic:                    100.92
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:05:48   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.9731     0.0813     73.482     0.0000      5.8137      6.1324
educ           0.0598     0.0060     10.046     0.00

In [182]:
## usando sibs como IV de educ
exog = None
endog = df[['educ']]
instrumentos = df[['sibs']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                     -0.0092
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0103
No. Observations:                 935   F-statistic:                    21.634
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:14:22   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.1300     0.3548     14.459     0.0000      4.4346      5.8254
educ           0.1224     0.0263     4.6513     0.00

In [183]:
## fazendo uma regressao simples de lwage sobre sibs
exog = df[['sibs']]
endog = None
instrumentos = None
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.0234
Estimator:                        OLS   Adj. R-squared:                 0.0223
No. Observations:                 935   F-statistic:                    22.355
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:14:24   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          6.8611     0.0221     311.10     0.0000      6.8179      6.9043
sibs          -0.0279     0.0059    -4.7281     0.00

In [184]:
equation()

O código da equação foi copiado para o clipboard!


In [185]:
## regredindo educ sobre sibs
exog = df[['sibs']]
endog = None
instrumentos = None
y = df['educ']
Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.0573
Estimator:                        OLS   Adj. R-squared:                 0.0562
No. Observations:                 935   F-statistic:                    56.789
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:14:46   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          14.139     0.1130     125.10     0.0000      13.917      14.360
sibs          -0.2279     0.0302    -7.5358     0.00

In [187]:
## regredindo educ sobre brthord (geralmente, por motivos orçamentários, apenas o primeiro filho pode ter verba pra ir a faculdade, por exemplo)
exog = df[['brthord']]
endog = None
instrumentos = None
y = df['educ']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.0420
Estimator:                        OLS   Adj. R-squared:                 0.0409
No. Observations:                 852   F-statistic:                    37.373
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:19:53   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          14.149     0.1285     110.09     0.0000      13.898      14.401
brthord       -0.2826     0.0462    -6.1134     0.00

In [189]:
## usando brthord como IV de educ
exog = None
endog = df[['educ']]
instrumentos = df[['brthord']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                     -0.0286
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0298
No. Observations:                 852   F-statistic:                    16.667
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:27:52   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.0304     0.4324     11.633     0.0000      4.1828      5.8780
educ           0.1306     0.0320     4.0825     0.00

In [190]:
## fazendo o teste de endogeneidade
Resultado.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 5.6569
P-value: 0.0174
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fb9402a9390

In [194]:
## usando brthord como IV de educ e controlando para sibs
exog = df[['sibs']]
endog = df[['educ']]
instrumentos = df[['brthord']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                     -0.0543
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0568
No. Observations:                 852   F-statistic:                    21.871
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:35:21   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          4.9385     1.0538     4.6863     0.0000      2.8731      7.0040
sibs           0.0021     0.0173     0.1217     0.90

In [193]:
## coletando os dados de primeiro estágio
print(Resultado.first_stage)

    First Stage Estimation Results    
                                  educ
--------------------------------------
R-squared                       0.0583
Partial R-squared               0.0084
Shea's R-squared                0.0084
Partial F-statistic             7.1804
P-value (Partial F-stat)        0.0074
Partial F-stat Distn           chi2(1)
========================== ===========
const                           14.296
                              (107.45)
sibs                           -0.1529
                             (-3.8409)
brthord                        -0.1527
                             (-2.6796)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [164]:
## coletando os dados da regressão de primeiro estágio
# criando um df droppando os valores nulos de brthord
df1 = df.dropna(subset=['brthord'], axis = 0)
x = df1[['sibs','brthord']]
y = df1['educ']

Regressao_Multipla(x,y)

## coletando os dados preditos
educ_chapeu = Resultado.fittedvalues

# vendo a correlação
df1['educ_chapeu'] = educ_chapeu
clear_output()
df1[['sibs','educ_chapeu']].corr()

,sibs,educ_chapeu
sibs,1.000000,-0.929482
educ_chapeu,-0.929482,1.000000


## Exercício C2

In [8]:
coletar_dados('FERTIL2')

FERTIL2.dta foi lido com sucesso!


,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,...,heduc,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr
0,5,64,24,1.0,1.0,1.0,1.0,12,0,NaN,...,NaN,576,1,12,0,0,0,1,0,0
1,1,56,32,1.0,1.0,1.0,1.0,13,3,25.0,...,12.0,1024,1,13,0,0,0,1,0,1
2,7,58,30,1.0,0.0,0.0,0.0,5,1,27.0,...,7.0,900,1,5,1,0,0,0,0,1
3,11,45,42,1.0,0.0,1.0,0.0,4,3,17.0,...,11.0,1764,1,4,0,0,0,0,0,1
4,5,45,43,1.0,1.0,1.0,1.0,11,2,24.0,...,14.0,1849,1,11,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,6,72,16,0.0,1.0,0.0,0.0,7,0,NaN,...,NaN,256,0,0,0,1,0,1,0,0
4357,1,60,28,0.0,1.0,0.0,0.0,7,2,19.0,...,NaN,784,0,0,0,1,0,1,0,0
4358,11,63,24,0.0,1.0,0.0,0.0,5,4,13.0,...,NaN,576,0,0,0,1,0,0,0,0
4359,1,62,26,0.0,1.0,0.0,0.0,0,1,25.0,...,7.0,676,0,0,1,0,0,1,1,1


In [196]:
## estimando o modelo proposto por MQO
exog = df[['educ','age','agesq']]
endog = None
instrumentos = None
y = df['children']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                            OLS Estimation Summary                            
Dep. Variable:               children   R-squared:                      0.5687
Estimator:                        OLS   Adj. R-squared:                 0.5684
No. Observations:                4361   F-statistic:                    5750.9
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:39:59   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -4.1383     0.2405    -17.208     0.0000     -4.6096     -3.6670
educ          -0.0906     0.0059    -15.305     0.00

In [200]:
## sendo frsthalf exógena, resta testar sua relevância
exog = df[['frsthalf','age','agesq']]
endog = None
instrumentos = None
y = df['educ']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.1077
Estimator:                        OLS   Adj. R-squared:                 0.1070
No. Observations:                4361   F-statistic:                    526.10
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        11:49:26   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.6929     0.5978     16.214     0.0000      8.5212      10.865
frsthalf      -0.8523     0.1128    -7.5572     0.00

In [20]:
## estimando o modelo proposto por MQ2E com frsthalf como vi de educ
exog = df[['age','agesq']]
endog = df[['educ']]
instrumentos = df[['frsthalf']]
y = df['children']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                          IV-2SLS Estimation Summary                          
Dep. Variable:               children   R-squared:                      0.5502
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5499
No. Observations:                4361   F-statistic:                    5300.2
Date:                Fri, Apr 02 2021   P-value (F-stat)                0.0000
Time:                        10:15:43   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -3.3878     0.5479    -6.1833     0.0000     -4.4617     -2.3139
age            0.3236     0.0179     18.128     0.00

In [76]:
## ajustando a equation() pra fazer tudo caber na linha do word
def equation(sep_erros= "("):
    '''
    Função que gera uma equação formatada do word
    '''
    ## Fazendo a str que irá pro word (em forma de matriz)
    inicio = "\matrix{"
    fim = "}"

    ## capturando os parametros, p-valores e os erros
    params = dict(np.around(Resultado.params,3))
    p_values = dict(np.around(Resultado.pvalues,4))

    ## linearmodels usa .std_erros para capturar os erros padrão, sm usa .bse
    try:
        std_errors = dict(np.around(Resultado.std_errors,4))
    except Exception:
        std_errors = dict(np.around(Resultado.bse,4))
    
    ## capturando as variáveis independentes indexadas por seu numero
    enum_params = dict(enumerate(params.keys()))
    enum_params = {value:key for key, value in enum_params.items()}
    
    ## pegando o nome da variável dependente
        # linearmodels usa model.dependent.dataframe.columns[0] para modelos de painel 
        # e model.dependent.cols para modelos de IV;
        # sm usa model.endog_names
    try:
        try: # modelos de painel
            y = Resultado.model.dependent.dataframe.columns[0]
        except Exception: # modelos de IV 2SLS
            y = Resultado.model.dependent.pandas.columns[0]
    except Exception: # modelos comuns
        y = Resultado.model.endog_names
    
    ## criando uma lista com ints até o número de linhas definido pelo usuário
    # temos que nos lembrar que no word só cabe ≈ 4 parâmetros por linha
        # math.ceil arredonda para cima; o + 1 é por conta de o python não considerar range como um intervalo fechado superiormente
    linhas = [num for num in range (1,math.ceil(len(Resultado.params)/4) + 1)]
    breaks = [4*num for num in range (1,math.ceil(len(Resultado.params)/4) + 1)]


    ## fazendo o loop para pegar os coeficientes*nome das variáveis e os seus erros-padrão entre o separador de erros
    parametros = ""
    erros = ""
    word = f"{inicio}{y} & = &"
    for i in params.keys():
        if enum_params[i] in breaks:
            ## adicionando tudo junto
            word += f"{parametros} \\\ & {erros} \\\ & "
            parametros = ""
            erros = ""
            # levando em conta a chave escolhida pelo usuário
            if sep_erros == "(":
                erros += f" & ({std_errors[i]})"
            else:
                erros += f" & [{std_errors[i]}]"
            # fazendo a string dos parâmetros:
                # *: p<0.1; **: p<0.05; ***: p<0.01
            if i != 'const':
                p = p_values[i]
                if params[i] > 0:
                    if p > 0.1:
                        parametros += f" & + {params[i]}{i}"
                    elif p < 0.01:
                        parametros += f" & + {params[i]}{i}^{{***}}"
                    elif p < 0.05:
                        parametros += f" & + {params[i]}{i}^{{**}}"
                    else:
                        parametros += f" & + {params[i]}{i}^{{*}}"
                else:
                    if p > 0.1:
                        parametros += f" & - {-params[i]}{i}"
                    elif p < 0.01:
                        parametros += f" & - {-params[i]}{i}^{{***}}"
                    elif p < 0.05:
                        parametros += f" & - {-params[i]}{i}^{{**}}"
                    else:
                        parametros += f" & - {-params[i]}{i}^{{*}}"
            else:
                parametros += f"{params[i]}"
        else:
            # levando em conta a chave escolhida pelo usuário
            if sep_erros == "(":
                erros += f" & ({std_errors[i]})"
            else:
                erros += f" & [{std_errors[i]}]"
            # fazendo a string dos parâmetros:
                # *: p<0.1; **: p<0.05; ***: p<0.01
            if i != 'const':
                p = p_values[i]
                if params[i] > 0:
                    if p > 0.1:
                        parametros += f" & + {params[i]}{i}"
                    elif p < 0.01:
                        parametros += f" & + {params[i]}{i}^{{***}}"
                    elif p < 0.05:
                        parametros += f" & + {params[i]}{i}^{{**}}"
                    else:
                        parametros += f" & + {params[i]}{i}^{{*}}"
                else:
                    if p > 0.1:
                        parametros += f" & - {-params[i]}{i}"
                    elif p < 0.01:
                        parametros += f" & - {-params[i]}{i}^{{***}}"
                    elif p < 0.05:
                        parametros += f" & - {-params[i]}{i}^{{**}}"
                    else:
                        parametros += f" & - {-params[i]}{i}^{{*}}"
            else:
                parametros += f"{params[i]}"

    word += f"{parametros} \\\ & {erros}"
    word += fim
    ## Adicionando o numero de obs e os r2 (quando rsquared_adj estiver disponível)
    try:
        word += f"\nn = {int(Resultado.nobs)}; R^2 = {np.around(Resultado.rsquared,4)}; \\bar{{R^2}} = {np.around(Resultado.rsquared_adj,4)}"
    except Exception:
        word += f"\nn = {int(Resultado.nobs)}; R^2 = {np.around(Resultado.rsquared,4)}"

    ## adicionando a explicação dos p-valores
    word += "\n^* p<0.1; ^{**} p<0.05; ^{***} p<0.01"
    ## substituindo os . por ,
    word = word.replace(".",",")
    
    ## copiando para o clipboard e printando o sucesso
    subprocess.run("pbcopy", universal_newlines=True, input=word)
    print("O código da equação foi copiado para o clipboard!")

In [80]:
## adicionando dummies exógenas e estimando por MQ2E
exog = df[['age','agesq','electric','tv','bicycle','yearborn','radio','usemeth','knowmeth']]
endog = df[['educ']]
instrumentos = df[['frsthalf']]
y = df['children']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                          IV-2SLS Estimation Summary                          
Dep. Variable:               children   R-squared:                      0.5012
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5000
No. Observations:                4282   F-statistic:                    4877.2
Date:                Fri, Apr 02 2021   P-value (F-stat)                0.0000
Time:                        11:16:47   Distribution:                 chi2(10)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          10.138     6.9202     1.4650     0.1429     -3.4252      23.702
age            0.0446     0.0993     0.4494     0.65

In [78]:
df.columns

Index(['mnthborn', 'yearborn', 'age', 'electric', 'radio', 'tv', 'bicycle',
       'educ', 'ceb', 'agefbrth', 'children', 'knowmeth', 'usemeth', 'monthfm',
       'yearfm', 'agefm', 'idlnchld', 'heduc', 'agesq', 'urban', 'urb_educ',
       'spirit', 'protest', 'catholic', 'frsthalf', 'educ0', 'evermarr'],
      dtype='object')

In [217]:
## vendo se as dummies sao relevantes
Teste_F_Rapido_Robusto('electric = tv = 0')

O valor de LM é 0.770748 e seu p-valor é 0.6801963.
Portanto, NÃO se rejeita Ho à significância de 5.0%, ou seja, as variáveis NÃO são conjuntamente significantes.


In [214]:
## adicionando dummies exógenas e estimando por MQ0
exog = df[['age','agesq','electric','tv','bicycle','educ']]
endog = None
instrumentos = None
y = df['children']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

/Users/vinicius/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning:


Inputs contain missing values. Dropping rows with missing observations.

                            OLS Estimation Summary                            
Dep. Variable:               children   R-squared:                      0.5761
Estimator:                        OLS   Adj. R-squared:                 0.5755
No. Observations:                4356   F-statistic:                    5919.0
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        12:02:30   Distribution:                  chi2(6)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------

## Exercício C3

In [219]:
coletar_dados('card')

card.dta foi lido com sucesso!


,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,sinmom14,...,smsa66,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq
0,2,0,0,7,29,NaN,NaN,158413.0,1,0,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,3,0,0,12,27,8.0,8.0,380166.0,1,0,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,4,0,0,12,34,14.0,12.0,367470.0,1,0,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256
3,5,1,1,11,27,11.0,12.0,380166.0,1,0,...,1,250,0,25.0,88.0,1.0,1.0,10,5.521461,100
4,6,1,1,12,34,8.0,7.0,367470.0,1,0,...,1,729,0,34.0,108.0,1.0,0.0,16,6.591674,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,5218,0,1,12,25,8.0,12.0,82135.0,1,0,...,0,335,0,15.0,NaN,1.0,0.0,7,5.814130,49
3006,5219,0,1,13,34,NaN,NaN,88765.0,1,0,...,0,481,0,43.0,NaN,1.0,1.0,15,6.175867,225
3007,5220,0,1,12,24,11.0,NaN,89271.0,0,0,...,0,500,0,25.0,109.0,1.0,0.0,6,6.214608,36
3008,5221,0,1,12,31,NaN,NaN,110376.0,1,0,...,0,713,0,32.0,107.0,1.0,1.0,13,6.569481,169


In [220]:
## regredindo IQ (proxy para aptidao) sobre nearc4
exog = df[['nearc4']]
endog = None
instrumentos = None
y = df['IQ']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                            OLS Estimation Summary                            
Dep. Variable:                     IQ   R-squared:                      0.0059
Estimator:                        OLS   Adj. R-squared:                 0.0054
No. Observations:                2061   F-statistic:                    12.140
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0005
Time:                        12:21:45   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          100.61     0.6272     160.42     0.0000      99.381      101.84
nearc4         2.5962     0.7451     3.4842     0.00

In [222]:
## regredindo IQ (proxy para aptidao) sobre nearc4 e outras variáveis geográficas
exog = df[['nearc4','smsa66','reg662','reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 'reg669']]
endog = None
instrumentos = None
y = df['IQ']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()

                            OLS Estimation Summary                            
Dep. Variable:                     IQ   R-squared:                      0.0626
Estimator:                        OLS   Adj. R-squared:                 0.0581
No. Observations:                2061   F-statistic:                    137.74
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        12:24:40   Distribution:                 chi2(10)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          104.77     1.6206     64.650     0.0000      101.60      107.95
nearc4         0.3479     0.8122     0.4283     0.66

## Exercício C5

In [7]:
coletar_dados('card')

card.dta foi lido com sucesso!


,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,sinmom14,...,smsa66,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq
0,2,0,0,7,29,NaN,NaN,158413.0,1,0,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,3,0,0,12,27,8.0,8.0,380166.0,1,0,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,4,0,0,12,34,14.0,12.0,367470.0,1,0,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256
3,5,1,1,11,27,11.0,12.0,380166.0,1,0,...,1,250,0,25.0,88.0,1.0,1.0,10,5.521461,100
4,6,1,1,12,34,8.0,7.0,367470.0,1,0,...,1,729,0,34.0,108.0,1.0,0.0,16,6.591674,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,5218,0,1,12,25,8.0,12.0,82135.0,1,0,...,0,335,0,15.0,NaN,1.0,0.0,7,5.814130,49
3006,5219,0,1,13,34,NaN,NaN,88765.0,1,0,...,0,481,0,43.0,NaN,1.0,1.0,15,6.175867,225
3007,5220,0,1,12,24,11.0,NaN,89271.0,0,0,...,0,500,0,25.0,109.0,1.0,0.0,6,6.214608,36
3008,5221,0,1,12,31,NaN,NaN,110376.0,1,0,...,0,713,0,32.0,107.0,1.0,1.0,13,6.569481,169


In [8]:
## regredindo educ sobre os controles da tabela 15.1 e coletando os resíduos v2
exog = df[['nearc4','exper','expersq','black','smsa','south','smsa66','reg662','reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 'reg669']]
endog = None
instrumentos = None
y = df['educ']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
v2 = Resultado.resids
df['v2'] = v2

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.4771
Estimator:                        OLS   Adj. R-squared:                 0.4745
No. Observations:                3010   F-statistic:                    2746.5
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        14:03:22   Distribution:                 chi2(15)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          16.638     0.2400     69.329     0.0000      16.168      17.109
nearc4         0.3199     0.0876     3.6506     0.00

In [9]:
## fazendo o teste de endogeneidade na mão
exog = df[['exper','expersq','black','smsa','south','smsa66','reg662','reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 'reg669','v2']]
endog = df[['educ']]
instrumentos = df[['nearc4']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
equation()
## como o p-valor de v2 é 0,2785, não rejeitamos h0: educ é exógeno

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.3001
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2964
No. Observations:                3010   F-statistic:                    1290.7
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        14:03:35   Distribution:                 chi2(16)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.6662     0.8841     4.1469     0.0000      1.9334      5.3989
exper          0.1083     0.0226     4.7874     0.00

In [230]:
## alternativamente, 
exog = df[['exper','expersq','black','smsa','south','smsa66','reg662','reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 'reg669']]
endog = df[['educ']]
instrumentos = df[['nearc4']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)
Resultado.wooldridge_regression

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.2382
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2343
No. Observations:                3010   F-statistic:                    769.20
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        12:45:08   Distribution:                 chi2(15)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.6662     0.9224     3.9747     0.0001      1.8583      5.4740
exper          0.1083     0.0236     4.5886     0.00

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 1.1743
P-value: 0.2785
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fb942a1f110

In [232]:
## fazendo o mesmo modelo mas adicionando nearc2 como instrumento
exog = df[['exper','expersq','black','smsa','south','smsa66','reg662','reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 'reg669']]
endog = df[['educ']]
instrumentos = df[['nearc4','nearc2']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1702
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1660
No. Observations:                3010   F-statistic:                    709.89
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        12:45:58   Distribution:                 chi2(15)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.2367     0.8826     3.6674     0.0002      1.5069      4.9665
exper          0.1188     0.0227     5.2237     0.00

In [233]:
## testando a sobreidentificacao das restricoes
Resultado.wooldridge_overid
## como o p-valor é 0,26, não se rejeita h0: os instrumentos são exógenos

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 1.2689
P-value: 0.2600
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fb9408e7dd0

## Exercício C9

In [234]:
coletar_dados('wage2')

wage2.dta foi lido com sucesso!


,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,520,40,79,28,16,6,1,30,1,1,1,0,0,1.0,11.0,NaN,6.253829
931,1202,40,102,32,13,10,3,31,1,0,1,1,7,7.0,8.0,6.0,7.091742
932,538,45,77,22,12,12,10,28,1,1,1,0,9,NaN,7.0,NaN,6.287858
933,873,44,109,25,12,12,12,28,1,0,1,0,1,1.0,NaN,11.0,6.771935


In [235]:
## fazendo o modelo por mq2e
exog = df[['exper','tenure','black']]
endog = df[['educ']]
instrumentos = df[['sibs']]
y = df['lwage']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1685
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1650
No. Observations:                 935   F-statistic:                    100.22
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        13:35:34   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.2160     0.5420     9.6236     0.0000      4.1537      6.2783
exper          0.0209     0.0084     2.5010     0.01

In [237]:
## fazendo a regressao de dois estagios na mão
x = df[['sibs','exper','tenure','black']]
y = df['educ']

Regressao_Multipla(x,y)
clear_output()

df['educ_chapeu'] = Lista_ychapeu

x = df[['exper','tenure','black','educ_chapeu']]
y = df['lwage']

Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     22.75
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           5.99e-18
Time:                        13:38:20   Log-Likelihood:                -474.00
No. Observations:                 935   AIC:                             958.0
Df Residuals:                     930   BIC:                             982.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.2160      0.569      9.170      

In [238]:
## fazendo a regressao de dois estagios na mão, sem levar em conta as outras variaveis exógenas do modelo
x = df[['sibs']]
y = df['educ']

Regressao_Multipla(x,y)
clear_output()

df['educ_chapeu'] = Lista_ychapeu

x = df[['exper','tenure','black','educ_chapeu']]
y = df['lwage']

Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     22.75
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           5.99e-18
Time:                        13:39:09   Log-Likelihood:                -474.00
No. Observations:                 935   AIC:                             958.0
Df Residuals:                     930   BIC:                             982.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.7710      0.360     16.014      

## Exercício C11

In [239]:
coletar_dados('voucher')

voucher.dta foi lido com sucesso!


,studyid,black,hispanic,female,appyear,mnce,select,choice,selectyrs,choiceyrs,mnce90,selectyrs1,selectyrs2,selectyrs3,selectyrs4,choiceyrs1,choiceyrs2,choiceyrs3,choiceyrs4
0,21,1,0,1,90,44,1,0,4,1,NaN,0,0,0,1,1,0,0,0
1,26,1,0,0,90,46,1,1,4,4,NaN,0,0,0,1,0,0,0,1
2,30,1,0,0,90,20,1,1,4,4,NaN,0,0,0,1,0,0,0,1
3,31,1,0,1,90,36,1,1,4,4,NaN,0,0,0,1,0,0,0,1
4,33,1,0,0,90,32,1,1,4,4,NaN,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,20708,1,0,0,93,34,1,0,1,0,NaN,1,0,0,0,0,0,0,0
986,20709,1,0,1,93,90,1,1,1,1,NaN,1,0,0,0,1,0,0,0
987,20717,1,0,0,93,34,1,1,1,1,NaN,1,0,0,0,1,0,0,0
988,20718,1,0,0,93,10,1,1,1,1,NaN,1,0,0,0,1,0,0,0


In [242]:
## vendo quantos estudantes nunca receberam um voucher
len(df.loc[df['selectyrs'] == 0])

468

In [243]:
## vendo quantos estudantes receberam um voucher por 4 anos
len(df.loc[df['selectyrs'] == 4])

108

In [244]:
## vendo quantos estudantes foram a uma escola por 4 anos
len(df.loc[df['choiceyrs'] == 4])

56

In [245]:
## fazendo uma regressao simples de choiceyrs sobre selectyrs
exog = df[['selectyrs']]
endog = None
instrumentos = None
y = df['choiceyrs']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

## dado que selectyrs é fruto de um sorteio em cada um dos anos, ela é exógena. os resultados dessa regressão simples mostra que, se não quisermos controlar para nenhum outro fator exógeno, selectyrs é relevante para explicar choiceyrs

                            OLS Estimation Summary                            
Dep. Variable:              choiceyrs   R-squared:                      0.7898
Estimator:                        OLS   Adj. R-squared:                 0.7896
No. Observations:                 990   F-statistic:                    3720.0
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        13:47:50   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0199     0.0246     0.8102     0.4179     -0.0283      0.0681
selectyrs      0.7668     0.0126     60.992     0.00

In [246]:
## fazendo uma regressao simples de mnce sobre choiceyrs
exog = df[['choiceyrs']]
endog = None
instrumentos = None
y = df['mnce']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

## definitivamente nao era o esperado: ter atendido escolas especiais por mais tempo diminui a nota do teste

                            OLS Estimation Summary                            
Dep. Variable:                   mnce   R-squared:                      0.0122
Estimator:                        OLS   Adj. R-squared:                 0.0112
No. Observations:                 990   F-statistic:                    12.243
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0005
Time:                        13:49:28   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          46.234     0.8498     54.403     0.0000      44.569      47.900
choiceyrs     -1.8370     0.5250    -3.4990     0.00

In [247]:
## fazendo uma regressao simples de mnce sobre choiceyrs e controlando para caracteristicas individuais
exog = df[['choiceyrs','black','hispanic','female']]
endog = None
instrumentos = None
y = df['mnce']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                            OLS Estimation Summary                            
Dep. Variable:                   mnce   R-squared:                      0.0868
Estimator:                        OLS   Adj. R-squared:                 0.0831
No. Observations:                 990   F-statistic:                    94.069
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        13:51:03   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          57.122     1.6525     34.567     0.0000      53.883      60.361
choiceyrs     -0.5652     0.5294    -1.0677     0.28

In [248]:
## contudo, choiceyrs pode ser endógeno, haja vista estar correlacionado com renda e até aptidão, ambas variáveis que afetam a nota de matemática
# usando uma VI para choiceyrs, o resultado do coeficiente ainda é negativo, mas bastante insignificante
# ser de uma cor marginalizada prejudica a nota, enquanto mulheres tiram, em média, notas maiores

exog = df[['black','hispanic','female']]
endog = df[['choiceyrs']]
instrumentos = df[['selectyrs']]
y = df['mnce']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   mnce   R-squared:                      0.0864
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0827
No. Observations:                 990   F-statistic:                    93.054
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        13:52:23   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          57.068     1.6535     34.513     0.0000      53.827      60.309
black         -16.317     1.8102    -9.0137     0.00

In [249]:
# controlando as notas de teste passada (≈ aptidão)
# AGORA CHOICEYRS FAZ SENTIDO, aumeentando em 1,8 a nota no teste
# contudo, o numero de observacoes cai bastante: pode haver uma autosseleção de pessoas mais dedicadas que fazem o exame em múltiplos anos

exog = df[['black','hispanic','female','mnce90']]
endog = df[['choiceyrs']]
instrumentos = df[['selectyrs']]
y = df['mnce']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   mnce   R-squared:                      0.4173
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4082
No. Observations:                 328   F-statistic:                    242.62
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        13:53:52   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          21.539     3.6120     5.9632     0.0000      14.459      28.618
black         -9.0671     2.5478    -3.5588     0.00

In [250]:
## fazendo uma regressao de iv usando as dummies
# ser selecionado para uma escola parece, de fato, não afetar a nota no teste (vide p-valores)
exog = df[['black','hispanic','female']]
endog = df[['choiceyrs1','choiceyrs2','choiceyrs3','choiceyrs4']]
instrumentos = df[['selectyrs1','selectyrs2','selectyrs3','selectyrs4']]
y = df['mnce']

Regressao_IV_MQ2E(exog= exog, endog= endog, instrumentos= instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   mnce   R-squared:                      0.0850
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0785
No. Observations:                 990   F-statistic:                    94.494
Date:                Sat, Mar 27 2021   P-value (F-stat)                0.0000
Time:                        13:57:16   Distribution:                  chi2(7)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          56.886     1.6631     34.205     0.0000      53.626      60.145
black         -16.297     1.8709    -8.7111     0.00